In [1]:
import os.path as pth
from scipy import io
from numba import jit
from scipy import stats
import nibabel as nib
import numpy as np
import datetime

In [2]:
@jit(nopython=True, cache=True, fastmath=True)
def make_sphere_array(radius):
    arr = np.full((2*radius+1,2*radius+1,2*radius+1), False)
    x0, y0, z0 = int(arr.shape[0]/2),int(arr.shape[1]/2),int(arr.shape[2]/2)
    for x in range(x0-radius, x0+radius+1):
        for y in range(y0-radius, y0+radius+1):
                for z in range(z0-radius, z0+radius+1):
                    deb = np.linalg.norm(np.array([x0-x, y0-y, z0-z], dtype=np.float32))
                    arr[x,y,z] = True if deb <= radius else False
    return arr

def search_light_analysis(data, radius, search_area=None, post_func=None):
    search_area = data if search_area is None else search_area 
    post_func = (lambda x:x) if post_func is None else post_func
    sphere_mask = make_sphere_array(radius)
    for x0 in range(radius, data.shape[0]-radius):
        for y0 in range(radius, data.shape[1]-radius):
            for z0 in range(radius, data.shape[2]-radius):
                if search_area[x0,y0,z0]:
                    target_area = data[x0-radius:x0+radius+1,y0-radius:y0+radius+1,z0-radius:z0+radius+1].copy()
                    available_mask = (target_area.sum(axis=3)!=0) & sphere_mask
                    if np.any(available_mask): # np.all : mask 아닌 영역이 있어도 계산 
                        values = target_area[available_mask]
                        result = post_func(values)
                        yield result, (x0,y0,z0)
                        
def load_brain_data(glm_result_path, run_index):
    run_str = 'r'+str(run_index+1).zfill(2)
    glm_result_path = pth.join(sbj_path, run_str, 'rst_sti_pc3_spmg1_re')
    head_filename = pth.join(glm_result_path, 'stats.afni_rst+tlrc.HEAD')
    head = nib.load(head_filename)
    return head.get_fdata()

In [3]:
mod = 'i'
data_base_path = pth.join('/data', '01_experiment_data', 'image_sound', 'prep_new_template')
label_list = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven',
                'eight', 'nine', 'bed', 'bird', 'cat', 'dog', 'house', 'tree']
affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [4]:
base = '/users/jmy/data/image_sound'
hcp = nib.load('/users/jmy/data/MNI_reference/hcp_360_3mm_64.nii').get_fdata()
hcp_bn = (hcp != 0).astype(int)

In [5]:
sbj_list = [25,26,29,30,31,32,33,34,37,38,39,40,41,43,44]
for sbj_idx in sbj_list:
    sbj_path = pth.join(data_base_path, 's'+str(sbj_idx))
    mat_file = io.loadmat(pth.join(sbj_path, 'stimuli', 'run_order.mat'))

    betas = np.zeros((64,76,64,192))
    idx_list = []
    idx2sortb = np.zeros((192),dtype=int)
    for runi in range(3):
        betas[:,:,:,64*runi:64*(runi+1)] = load_brain_data(sbj_path, runi)[:,:,:,1:][:,:,:,[3*i for i in range(192)]][:,:,:,64:128]
        key_ = 'run'+str(runi+1)
        idx_list += [v.strip() for v in mat_file[key_] if v.startswith('i_')]
    idx_arr = np.array(idx_list)
    for li, label in enumerate(label_list):
        label_idx = [i for i,v in enumerate(idx_arr) if v[2:].startswith(label)]
        sort_idx = np.argsort(np.array([v[2+len(label):] for v in idx_arr[label_idx]],dtype=int))
        idx2sortb[li*12:(li+1)*12] = np.array(label_idx)[sort_idx]
    betas = betas[:,:,:,(idx2sortb)]
    beta_msk = np.where(np.sum((betas!=0),axis=3)==192)
    beta_vec = betas[beta_msk] # nvv,192
    for voli in range(192): # zscore
        beta_vec[:,voli] = stats.zscore(beta_vec[:,voli])
    down_vec = np.zeros((beta_vec.shape[0],16),)
    for ci in range(16):
        tmpvecs = beta_vec[:,ci*12:(ci+1)*12]
        down_vec[:,ci] = np.mean(tmpvecs,axis=1)
    down_betas = np.zeros((64,76,64,16))
    down_betas[beta_msk] = down_vec

    mask = hcp_bn.copy()
    tmp = np.sum((down_betas!=0),axis=3)
    mask[np.where(tmp<16)] = 0 

    brain_inform_list = [inform for inform in search_light_analysis(down_betas,radius=3,search_area=mask)]
    nvv = len(brain_inform_list)
    brain_rdm = np.zeros((nvv,16,16))
    cnt_info = np.zeros((nvv),dtype=int)
    for vi in range(nvv): 
        tp = brain_inform_list[vi][0] # ?,192
        tp2 = 1-np.corrcoef(tp, rowvar=False)
        brain_rdm[vi] = tp2
        cnt_info[vi] = tp.shape[0]
    #save brain_rdm
    savepath = base+'/RDMs/Neural/16x16/P'+str(sbj_idx)+'_{}.npz'.format(mod)
    np.savez(savepath, **{'rdm':brain_rdm, 'info_list':brain_inform_list})
    print(sbj_idx,'MIN:',cnt_info.min(),'MEAN: %.2f'%cnt_info.mean(),'STD: %.2f'%cnt_info.std(),np.where(cnt_info<100)[0].shape[0],'less than 100 AMONG', nvv)

25 MIN: 42 MEAN: 110.34 STD: 18.76 8218 less than 100 AMONG 33996
26 MIN: 37 MEAN: 110.34 STD: 18.86 8008 less than 100 AMONG 32972
29 MIN: 41 MEAN: 110.36 STD: 18.86 7891 less than 100 AMONG 32611
30 MIN: 38 MEAN: 110.31 STD: 18.86 8032 less than 100 AMONG 33199
31 MIN: 40 MEAN: 110.40 STD: 18.80 7984 less than 100 AMONG 33130
32 MIN: 37 MEAN: 110.48 STD: 18.72 8023 less than 100 AMONG 33576
33 MIN: 38 MEAN: 110.32 STD: 18.78 8232 less than 100 AMONG 33942
34 MIN: 32 MEAN: 110.16 STD: 18.98 8061 less than 100 AMONG 32742
37 MIN: 40 MEAN: 110.34 STD: 18.82 8039 less than 100 AMONG 33334
38 MIN: 42 MEAN: 110.43 STD: 18.73 8166 less than 100 AMONG 34123
39 MIN: 39 MEAN: 110.32 STD: 18.84 8074 less than 100 AMONG 33417
40 MIN: 38 MEAN: 110.28 STD: 18.85 8195 less than 100 AMONG 33739
41 MIN: 42 MEAN: 110.37 STD: 18.78 8176 less than 100 AMONG 33926
43 MIN: 37 MEAN: 110.41 STD: 18.79 7955 less than 100 AMONG 33082
44 MIN: 31 MEAN: 110.31 STD: 18.93 7793 less than 100 AMONG 32253
